In [1]:
from dotenv import load_dotenv
import os
import openai

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
# selenium 4
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time 
from bs4 import BeautifulSoup


driver = webdriver.Chrome(ChromeDriverManager().install())

driver.get('https://19hz.info/eventlisting_BayArea.php#crewList')
driver.find_element_by_xpath('/html/body/div[1]/div/span[1]/button[1]').click()
time.sleep(5)
body = driver.find_element_by_xpath('/html/body')

events = []
recurring_events = []
table = driver.find_element(By.XPATH, '/html/body/table[1]/tbody')
rows = table.find_elements(By.TAG_NAME, "tr") # get all of the rows in the table
for row in rows:
    events.append(row.text)

rec_table = driver.find_element_by_id("mobileEnable")
rows = rec_table.find_elements(By.TAG_NAME, "tr") # get all of the rows in the table
for row in rows:
    recurring_events.append(row.text)


[WDM] - Downloading: 100%|██████████| 8.82M/8.82M [00:00<00:00, 24.7MB/s]


In [3]:
text = body.text.split('\n')
start_index,end_index = 0,0
for i in range(len(text)):
    if text[i] == 'Venue List':
        start_index = i+2
    if text[i] == 'Promoter List':
        end_index = i
        break
text_comp = text[start_index:end_index]
venues = {}
for line in text_comp:
    name,address = line.split('-')
    name = name.rstrip()
    address = address.lstrip()
    venues[name] = address
driver.quit()

In [4]:
from opencage.geocoder import OpenCageGeocode
from shapely.geometry import Point


key = '21b734acde524209895d1c01fc8e6eea'
geocoder = OpenCageGeocode(key)

In [5]:
def generate_prompt(event):
    prompt = """Extract the Date/Time, Event Name, Venue, Music Genres, Price, Age Requirement, and Performers from each event and store it into a JSON with the following structure {venue: venue_name, event: event_name, date: date_and_time, age: age_requirement, price: price, music:[musc genres], performers:[performers]}""" + '\n' + "Event:" + event + '\n' + 'Enclose each key and value in the JSON structure with double quotes.'
    return prompt

def find_address(venue):
    prompt = "Find the full address for the San Francisco venue." + '\n' + 'Venue:' + venue + '\n' 
    return prompt

In [32]:
response = openai.Completion.create(
        model="text-davinci-003",
        prompt='What is the date exactly 15 from today? Provide a text format such as "Feb 22" where the month is abbreviated.',
        temperature=0.0,
        max_tokens = 50,
  )
stop_date = response['choices'][0]['text'].strip()
print(stop_date)

Mar 8


In [30]:
import json
import pandas as pd
import numpy as np

sf_events = pd.DataFrame()


for event in events:
  response = openai.Completion.create(
        model="text-davinci-003",
        prompt=generate_prompt(event),
        temperature=0.0,
        max_tokens = 200,
  )
  response_text = response['choices'][0]['text'].strip()
  try:
    response_dict = json.loads(response_text)
  except:
    print(response_text)    
    break

  venue = response_dict['venue']

  time.sleep(2)
  response = openai.Completion.create(
      model="text-curie-001",
      prompt=find_address(venue),
      temperature=0.0,
      max_tokens = 100,
  )

  location = response['choices'][0]['text'].strip()
  response_dict['location'] = location

  results = geocoder.geocode(location)
  try:
    loc = Point(results[0]['geometry']['lat'],results[0]['geometry']['lng'])
    response_dict['geolocation'] = loc
  except:
    response_dict['geolocation'] = np.nan
  
  if stop_date in response_dict['date']:
    break
 
  sf_events = sf_events.append(response_dict, ignore_index=True)

/Users/gauravmohan/opt/anaconda3/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


KeyboardInterrupt: 

In [31]:
sf_events

,venue,event,date,age,price,music,performers,location,geolocation
0,1015 Folsom (San Francisco),Good Fun Luv n More,Wed: Feb 22 (6pm-10pm),21+,$10-15,[house],[The Good Fun Event Link],"1015 Folsom Street\nSan Francisco, CA 94103",POINT (37.7780411 -122.405683)
1,Mothership Bar,Local / Global,Wed: Feb 22 (8pm-12am),21+,free,"[house, techno, global beats]",[DJmaDRE],The Mothership Bar is located at 333 Bush Stre...,POINT (32.38141 76.51194)
2,The Fillmore,The Chainsmokers,"Friday, April 10th, 2020 at 8:00 PM",18+,$50.00,"[EDM, Pop]",[The Chainsmokers],The Fillmore is located at 1805 Geary Street i...,POINT (37.77493 -122.41942)
3,F8 1192 Folsom,"John Tejada, Ghenji, Michael May",Wed: Feb 22 (9pm-2am),21+,free w/rsvp b4 11pm/$10,"[deep house, funky house]","[Strut SF, Delta Funk]",The address for the San Francisco venue is 119...,NaN
4,Monarch,Soundpieces Ft Hypho | Hebbe | Drezza | Heisty...,Wed: Feb 22 (9pm-2am),21+,$10-18,midtempo bass,Hypho | Hebbe | Drezza | Heisty X Alt-1,The Monarch is located at 3170 Mission Street ...,POINT (37.77493 -122.41942)
...,...,...,...,...,...,...,...,...,...
184,The Midway,Krs-One,Fri: Mar 24 (8pm-12am),21+,$20-30,[hip-hop],[Krs-One],The Midway is located at 333 Bush Street in Sa...,POINT (37.77493 -122.41942)
185,The Independent,Minnesota,Fri: Mar 24 (9pm),21+,$22,"[space bass, dubstep, hybrid trap]",[The Independent Facebook Page],The Independent is located at 3170 Mission Str...,POINT (37.77493 -122.41942)
186,DNA Lounge,Dystopia Dnb 12-Year Anniversary,Fri: Mar 24 (9pm-2:30am),21+,$10-15 pre/$25,"[drum and bass, jungle, electro house]","[Sub Killaz, Whys, Blunter S. Whompson, DJ W4X...",DNA Lounge is located at 333 Bush Street in Sa...,POINT (32.38141 76.51194)
187,Public Works,"Rodhad, Noncompliant, Max Gardner b2b Mossmoss",Fri: Mar 24 (9pm-3am),21+,$15-25,[techno],"[As You Like It, Direct To Earth]",The address for the San Francisco venue is:\n\...,POINT (37.786248 -122.402025)


## Get Embeddings

In [33]:
import pandas as pd
import tiktoken
from openai.embeddings_utils import get_embedding
import os
import openai

openai.apikey = os.getenv("OPENAI_API_KEY")
# embedding model parameters
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"  # this the encoding for text-embedding-ada-002
max_tokens = 5000  # the maximum for text-embedding-ada-002 is 8191

In [34]:
def find_venue_type(venue):
    prompt = f'Provide a short description of the following venue in SF? Utilize the venues website or online search to find the answer.' + '\n' + 'Venue: ' + venue
    return prompt

def find_neighborhood(venue):
    prompt = "Given the following venue, provide the SF neighborhood that this venue is located in." + '\n' + 'Venue: ' + venue
    return prompt

In [35]:
venue_description = {}
for index, row in sf_events.iterrows():
    if row['venue'] not in venue_description:
        response = openai.Completion.create(
            model="text-curie-001",
            prompt=find_venue_type(row['venue']),
            temperature=0.3,
            max_tokens = 100,
        )
        response_text = response['choices'][0]['text'].strip()
        venue_description[row['venue']] = response_text
        sf_events.loc[index, 'venue_description'] = response_text

    else:
        sf_events.loc[index, 'venue_description'] = venue_description[row['venue']]    


In [36]:
venue_description = {}
for index, row in sf_events.iterrows():
    if row['venue'] not in venue_description:
        response = openai.Completion.create(
            model="text-curie-001",
            prompt=find_neighborhood(row['venue']),
            temperature=0.0,
            max_tokens = 50,
        )
        response_text = response['choices'][0]['text'].strip()
        venue_description[row['venue']] = response_text
        sf_events.loc[index, 'neighborhood'] = response_text

    else:
        sf_events.loc[index, 'neighborhood'] = venue_description[row['venue']]  

In [39]:
df_embeddings = sf_events.applymap(str)
encoding = tiktoken.get_encoding(embedding_encoding)
sf_events["combined"] = (
    "Venue: " + df_embeddings.venue.str.strip() + "; Venue Description: " + df_embeddings.venue_description.str.strip() + 
    "; Event: " + df_embeddings.event.str.strip() +  "; Event Description: " + df_embeddings.music.str.strip() + ' ' + df_embeddings.performers.str.strip() + 
    "; Price: " + df_embeddings.price.str.strip() +  "; Neighborhood: " + df_embeddings.neighborhood.str.strip()
)

sf_events["n_tokens"] = sf_events.combined.apply(lambda x: len(encoding.encode(x)))

In [40]:
sf_events["embedding"] = sf_events.combined.apply(lambda x: get_embedding(x, engine=embedding_model))
sf_events.to_csv("19hz_with_embeddings.csv")